In [118]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
import re as re
from opencage.geocoder import OpenCageGeocode
from geopy.distance import geodesic
import requests

sb.set() # set the default Seaborn style for graphics

In [102]:
flightData = pd.read_csv('https://raw.githubusercontent.com/chongwei49/IE0005-DSAI-Group-Barley/master/dataset/economy.csv')
flightData.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11/2/2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11/2/2022,SpiceJet,SG,8157,6:20,Delhi,02h 20m,non-stop,8:40,Mumbai,"5,953"
2,11/2/2022,AirAsia,I5,764,4:25,Delhi,02h 10m,non-stop,6:35,Mumbai,"5,956"
3,11/2/2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11/2/2022,Vistara,UK,963,8:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


In [103]:
flightData.drop(columns=['ch_code', 'num_code'], axis=1, inplace=True) #Remove 'ch_code' and 'num_code' column

flightData['stop'] = flightData['stop'].str.split(r'\n').str.get(0) #To clean 'stop' column

flightData['price'] = flightData['price'].str.replace(',', '') #To clean 'price' column

flightData['dep_time'] = flightData['dep_time'].str.replace(':', '') #To clean 'dep_time' column
flightData['dep_time'] = flightData['dep_time'].str.zfill(4) #To show time in HHMM format

flightData['arr_time'] = flightData['arr_time'].str.replace(':', '') #To clean 'arr_time' column
flightData['arr_time'] = flightData['arr_time'].str.zfill(4) #To show time in HHMM format

In [104]:
flightData.dtypes

date          object
airline       object
dep_time      object
from          object
time_taken    object
stop          object
arr_time      object
to            object
price         object
dtype: object

In [105]:
price = pd.DataFrame(flightData['price'])
print("Data type : ", type(price))
print("Data dims : ", price.size)
price.head()

Data type :  <class 'pandas.core.frame.DataFrame'>
Data dims :  206774


,price
0,5953
1,5953
2,5956
3,5955
4,5955


In [106]:
price.describe()

,price
count,206774
unique,9819
top,2339
freq,1442


In [107]:
#   pattern = re.compile(r'^([0-9]{2}[h])\s([0-9]{2}[m])')
#   for time in flightData['time_taken']:
#       time = str(time)
#       for h,m in re.findall(pattern, time):
       
#           x = int(h[slice(0,2)])
#           y = int(m[slice(0,2)])
#           x = x*60
#     # print(time)        
#       # print(x+y)
#       flightData['time_taken'] = flightData['time_taken'].replace(time,x+y) #To clean 'time_taken' column
#           # y = m.str.slice(0,1)
# flightData
         

In [108]:
# refine above code
# 

flightData['time_taken'] = flightData['time_taken'].str.replace('h','')
flightData['time_taken'] = flightData['time_taken'].str.replace('m','')
flightData['time_taken'] = (pd.to_numeric(flightData['time_taken'].str.split(' ').str.get(0), downcast="integer")*60) + pd.to_numeric(flightData['time_taken'].str.split(' ').str.get(1), downcast="integer")

flightData_trim = flightData[:1000] #Trim data down to 1000 rows instead of its full dataset which has 206774 rows (to prevent lagging)
flightData_trim #Display trimmed data
         

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price
0,11/2/2022,SpiceJet,1855,Delhi,130.0,non-stop,2105,Mumbai,5953
1,11/2/2022,SpiceJet,0620,Delhi,140.0,non-stop,0840,Mumbai,5953
2,11/2/2022,AirAsia,0425,Delhi,130.0,non-stop,0635,Mumbai,5956
3,11/2/2022,Vistara,1020,Delhi,135.0,non-stop,1235,Mumbai,5955
4,11/2/2022,Vistara,0850,Delhi,140.0,non-stop,1110,Mumbai,5955
...,...,...,...,...,...,...,...,...,...
995,17/2/2022,Air India,0535,Delhi,910.0,1-stop,2045,Mumbai,9000
996,17/2/2022,Air India,0515,Delhi,1070.0,1-stop,2305,Mumbai,9000
997,17/2/2022,Air India,1900,Delhi,1250.0,1-stop,1550,Mumbai,9053
998,17/2/2022,Indigo,2150,Delhi,625.0,1-stop,0815,Mumbai,9263


In [109]:
flightData_trim.dtypes

date           object
airline        object
dep_time       object
from           object
time_taken    float64
stop           object
arr_time       object
to             object
price          object
dtype: object

In [110]:
state = []
for s in flightData['from'].unique():
  key = '6280b52e2b134b1a8d709f1fa15464fd'
  geocoder = OpenCageGeocode(key)
  result = geocoder.geocode(s)
  lat = result[0]['geometry']['lat']
  lng = result[0]['geometry']['lng']
  state.append([s, lat, lng]) # state[0]: name, state[1]: latitude, state[2]: longtitude

print(state)

[['Delhi', 28.6517178, 77.2219388], ['Mumbai', 19.0785451, 72.878176], ['Bangalore', 12.9767936, 77.590082], ['Kolkata', 22.5414185, 88.3576912], ['Hyderabad', 17.360589, 78.4740613], ['Chennai', 13.0836939, 80.270186]]


In [111]:
def add_from_lat(row): #Function to get latitude
  for s in state:
    if row['from'] == s[0]:
      return s[1]

def add_from_lng(row): #Function to get longtitude
  for s in state:
    if row['from'] == s[0]:
      return s[2]

flightData["from_lat"] = flightData.apply(add_from_lat, axis=1)
flightData["from_lng"] = flightData.apply(add_from_lng, axis=1)
flightData

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,from_lat,from_lng
0,11/2/2022,SpiceJet,1855,Delhi,130.0,non-stop,2105,Mumbai,5953,28.651718,77.221939
1,11/2/2022,SpiceJet,0620,Delhi,140.0,non-stop,0840,Mumbai,5953,28.651718,77.221939
2,11/2/2022,AirAsia,0425,Delhi,130.0,non-stop,0635,Mumbai,5956,28.651718,77.221939
3,11/2/2022,Vistara,1020,Delhi,135.0,non-stop,1235,Mumbai,5955,28.651718,77.221939
4,11/2/2022,Vistara,0850,Delhi,140.0,non-stop,1110,Mumbai,5955,28.651718,77.221939
...,...,...,...,...,...,...,...,...,...,...,...
206769,31/3/2022,Vistara,0705,Chennai,830.0,1-stop,2055,Hyderabad,7697,13.083694,80.270186
206770,31/3/2022,Vistara,0705,Chennai,830.0,1-stop,2055,Hyderabad,7709,13.083694,80.270186
206771,31/3/2022,Vistara,1230,Chennai,1235.0,1-stop,0905,Hyderabad,8640,13.083694,80.270186
206772,31/3/2022,Vistara,0945,Chennai,1400.0,1-stop,0905,Hyderabad,8640,13.083694,80.270186


In [112]:
def add_to_lat(row): #Function to get latitude
  for s in state:
    if row['to'] == s[0]:
      return s[1]

def add_to_lng(row): #Function to get longtitude
  for s in state:
    if row['to'] == s[0]:
      return s[2]

flightData["to_lat"] = flightData.apply(add_to_lat, axis=1)
flightData["to_lng"] = flightData.apply(add_to_lng, axis=1)
flightData

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,from_lat,from_lng,to_lat,to_lng
0,11/2/2022,SpiceJet,1855,Delhi,130.0,non-stop,2105,Mumbai,5953,28.651718,77.221939,19.078545,72.878176
1,11/2/2022,SpiceJet,0620,Delhi,140.0,non-stop,0840,Mumbai,5953,28.651718,77.221939,19.078545,72.878176
2,11/2/2022,AirAsia,0425,Delhi,130.0,non-stop,0635,Mumbai,5956,28.651718,77.221939,19.078545,72.878176
3,11/2/2022,Vistara,1020,Delhi,135.0,non-stop,1235,Mumbai,5955,28.651718,77.221939,19.078545,72.878176
4,11/2/2022,Vistara,0850,Delhi,140.0,non-stop,1110,Mumbai,5955,28.651718,77.221939,19.078545,72.878176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
206769,31/3/2022,Vistara,0705,Chennai,830.0,1-stop,2055,Hyderabad,7697,13.083694,80.270186,17.360589,78.474061
206770,31/3/2022,Vistara,0705,Chennai,830.0,1-stop,2055,Hyderabad,7709,13.083694,80.270186,17.360589,78.474061
206771,31/3/2022,Vistara,1230,Chennai,1235.0,1-stop,0905,Hyderabad,8640,13.083694,80.270186,17.360589,78.474061
206772,31/3/2022,Vistara,0945,Chennai,1400.0,1-stop,0905,Hyderabad,8640,13.083694,80.270186,17.360589,78.474061


In [113]:
def cal_distance(row): #Calculate the distance based on latitude and longtitude of from and to
  return (geodesic((row['from_lat'],row['from_lng']), (row['to_lat'],row['to_lng'])).km)

flightData["distance"] = flightData.apply(cal_distance, axis=1) 
flightData

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,from_lat,from_lng,to_lat,to_lng,distance
0,11/2/2022,SpiceJet,1855,Delhi,130.0,non-stop,2105,Mumbai,5953,28.651718,77.221939,19.078545,72.878176,1148.593810
1,11/2/2022,SpiceJet,0620,Delhi,140.0,non-stop,0840,Mumbai,5953,28.651718,77.221939,19.078545,72.878176,1148.593810
2,11/2/2022,AirAsia,0425,Delhi,130.0,non-stop,0635,Mumbai,5956,28.651718,77.221939,19.078545,72.878176,1148.593810
3,11/2/2022,Vistara,1020,Delhi,135.0,non-stop,1235,Mumbai,5955,28.651718,77.221939,19.078545,72.878176,1148.593810
4,11/2/2022,Vistara,0850,Delhi,140.0,non-stop,1110,Mumbai,5955,28.651718,77.221939,19.078545,72.878176,1148.593810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206769,31/3/2022,Vistara,0705,Chennai,830.0,1-stop,2055,Hyderabad,7697,13.083694,80.270186,17.360589,78.474061,511.055118
206770,31/3/2022,Vistara,0705,Chennai,830.0,1-stop,2055,Hyderabad,7709,13.083694,80.270186,17.360589,78.474061,511.055118
206771,31/3/2022,Vistara,1230,Chennai,1235.0,1-stop,0905,Hyderabad,8640,13.083694,80.270186,17.360589,78.474061,511.055118
206772,31/3/2022,Vistara,0945,Chennai,1400.0,1-stop,0905,Hyderabad,8640,13.083694,80.270186,17.360589,78.474061,511.055118


In [114]:
def modify_stop(row):
  if "non-stop" in row['stop']:
    return 0
  else:
    return int(re.search(r'\d+', row['stop']).group())

flightData["stop"] = flightData.apply(modify_stop, axis=1) 
flightData

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,from_lat,from_lng,to_lat,to_lng,distance
0,11/2/2022,SpiceJet,1855,Delhi,130.0,0,2105,Mumbai,5953,28.651718,77.221939,19.078545,72.878176,1148.593810
1,11/2/2022,SpiceJet,0620,Delhi,140.0,0,0840,Mumbai,5953,28.651718,77.221939,19.078545,72.878176,1148.593810
2,11/2/2022,AirAsia,0425,Delhi,130.0,0,0635,Mumbai,5956,28.651718,77.221939,19.078545,72.878176,1148.593810
3,11/2/2022,Vistara,1020,Delhi,135.0,0,1235,Mumbai,5955,28.651718,77.221939,19.078545,72.878176,1148.593810
4,11/2/2022,Vistara,0850,Delhi,140.0,0,1110,Mumbai,5955,28.651718,77.221939,19.078545,72.878176,1148.593810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206769,31/3/2022,Vistara,0705,Chennai,830.0,1,2055,Hyderabad,7697,13.083694,80.270186,17.360589,78.474061,511.055118
206770,31/3/2022,Vistara,0705,Chennai,830.0,1,2055,Hyderabad,7709,13.083694,80.270186,17.360589,78.474061,511.055118
206771,31/3/2022,Vistara,1230,Chennai,1235.0,1,0905,Hyderabad,8640,13.083694,80.270186,17.360589,78.474061,511.055118
206772,31/3/2022,Vistara,0945,Chennai,1400.0,1,0905,Hyderabad,8640,13.083694,80.270186,17.360589,78.474061,511.055118


In [123]:
class RealTimeCurrencyConverter(): #CurrencyConverter class which will get the real-time exchange rate and convert the currency and return the converted amount.
    def __init__(self,url):
        self.data= requests.get(url).json() #requests.get(url) load the page in our python program and then .json() will convert the page into the json file. We store it in a data variable.
        self.currencies = self.data['conversion_rates']

    def convert(self, from_currency, to_currency, amount): 
        initial_amount = amount 
        #first convert it into USD if it is not in USD.
        # because our base currency is USD
        if from_currency != 'USD' : 
            amount = amount / self.currencies[from_currency] 
      
        # limiting the precision to 4 decimal places 
        amount = round(amount * self.currencies[to_currency], 4) 
        return amount


url = 'https://v6.exchangerate-api.com/v6/353b9f00085a8c80b5133637/latest/USD'
converter = RealTimeCurrencyConverter(url)
print(converter.convert('SGD','INR',100))

5773.8351


In [128]:
def cal_price_sg(row):
  return converter.convert('INR','SGD',float(row['price']))

flightData["price_sg"] = flightData.apply(cal_price_sg, axis=1) 
flightData

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,from_lat,from_lng,to_lat,to_lng,distance,price_sg
0,11/2/2022,SpiceJet,1855,Delhi,130.0,0,2105,Mumbai,5953,28.651718,77.221939,19.078545,72.878176,1148.593810,103.1030
1,11/2/2022,SpiceJet,0620,Delhi,140.0,0,0840,Mumbai,5953,28.651718,77.221939,19.078545,72.878176,1148.593810,103.1030
2,11/2/2022,AirAsia,0425,Delhi,130.0,0,0635,Mumbai,5956,28.651718,77.221939,19.078545,72.878176,1148.593810,103.1550
3,11/2/2022,Vistara,1020,Delhi,135.0,0,1235,Mumbai,5955,28.651718,77.221939,19.078545,72.878176,1148.593810,103.1377
4,11/2/2022,Vistara,0850,Delhi,140.0,0,1110,Mumbai,5955,28.651718,77.221939,19.078545,72.878176,1148.593810,103.1377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206769,31/3/2022,Vistara,0705,Chennai,830.0,1,2055,Hyderabad,7697,13.083694,80.270186,17.360589,78.474061,511.055118,133.3083
206770,31/3/2022,Vistara,0705,Chennai,830.0,1,2055,Hyderabad,7709,13.083694,80.270186,17.360589,78.474061,511.055118,133.5161
206771,31/3/2022,Vistara,1230,Chennai,1235.0,1,0905,Hyderabad,8640,13.083694,80.270186,17.360589,78.474061,511.055118,149.6406
206772,31/3/2022,Vistara,0945,Chennai,1400.0,1,0905,Hyderabad,8640,13.083694,80.270186,17.360589,78.474061,511.055118,149.6406


In [130]:
flightData.dtypes

date           object
airline        object
dep_time       object
from           object
time_taken    float64
stop            int64
arr_time       object
to             object
price          object
from_lat      float64
from_lng      float64
to_lat        float64
to_lng        float64
distance      float64
price_sg      float64
dtype: object